In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [5]:
DATASET_PATH = 'Task_B/train'
DATASET_PATH1 = 'Task_B/val'
BATCH_SIZE = 16
EPOCHS = 10
EMBEDDING_DIM = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Face detector (MTCNN) and frozen FaceNet (InceptionResnetV1)
mtcnn = MTCNN(image_size=160, margin=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
for param in resnet.parameters():
    param.requires_grad = False  # Freeze for now

In [7]:
class FaceDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        self.label_map = {}
        self._load_data(root_dir)

    def _load_data(self, root_dir):
        label_index = 0
        for person in sorted(os.listdir(root_dir)):
            person_path = os.path.join(root_dir, person)
            if not os.path.isdir(person_path):
                continue

            if person not in self.label_map:
                self.label_map[person] = label_index
                label_index += 1
            label = self.label_map[person]

            # Original image
            original_img = os.path.join(person_path, f"{person}.jpg")
            if os.path.exists(original_img):
                self.samples.append((original_img, label))

            # Distorted images
            distortion_dir = os.path.join(person_path, "distortion")
            if os.path.isdir(distortion_dir):
                for img_name in os.listdir(distortion_dir):
                    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        self.samples.append((os.path.join(distortion_dir, img_name), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is None:
            # Retry next index if face not found
            return self.__getitem__((idx + 1) % len(self.samples))
        with torch.no_grad():
            embedding = resnet(face.unsqueeze(0).to(device)).squeeze(0).cpu()
        return embedding, label


In [8]:
class ClassifierHead(nn.Module):
    def __init__(self, embedding_dim=512, num_classes=10):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

In [9]:
# Load dataset and model
train_dataset = FaceDataset(DATASET_PATH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


val_dataset = FaceDataset(DATASET_PATH1)
val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

num_classes = len(train_dataset.label_map)
classifier = ClassifierHead(embedding_dim=EMBEDDING_DIM, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=1e-3)


In [15]:
for epoch in range(EPOCHS):
    classifier.train()
    total_loss = 0
    correct = 0
    total = 0

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    for embeddings, labels in tqdm(train_loader, desc='Training'):
        embeddings, labels = embeddings.to(device), labels.to(device)

        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Validation Step
    classifier.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for embeddings, labels in tqdm(val_loader, desc='Validation'):
            embeddings, labels = embeddings.to(device), labels.to(device)
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

    # Early Stopping Check
    best_val_acc = 0
    patience = 1         # You can increase this if needed
    counter = 0

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        torch.save(classifier.state_dict(), 'best_classifier.pth')
        print(f"✅ New best model saved (Val Acc: {val_acc:.2f}%)")
    else:
        counter += 1
        print(f"⏳ No improvement for {counter} epoch(s)")

    if counter >= patience:
        print("⛔ Early stopping triggered.")
        break



Epoch 1/10


Validation: 100%|████████████████████████████████████████████████████████████████████| 187/187 [08:15<00:00,  2.65s/it]


Train Loss: 1.2453 | Train Acc: 79.51%
Val   Loss: 0.6803 | Val   Acc: 91.68%
✅ New best model saved (Val Acc: 91.68%)

Epoch 2/10


Validation: 100%|████████████████████████████████████████████████████████████████████| 187/187 [12:12<00:00,  3.91s/it]


Train Loss: 0.7013 | Train Acc: 89.67%
Val   Loss: 0.4247 | Val   Acc: 93.59%
✅ New best model saved (Val Acc: 93.59%)

Epoch 3/10


...

Train Loss: 0.2297 | Train Acc: 95.51%
Val   Loss: 0.1285 | Val   Acc: 97.75%
✅ New best model saved (Val Acc: 97.75%)

Epoch 9/10


Validation: 100%|████████████████████████████████████████████████████████████████████| 187/187 [06:21<00:00,  2.04s/it]


Train Loss: 0.2098 | Train Acc: 96.04%
Val   Loss: 0.1091 | Val   Acc: 98.26%
✅ New best model saved (Val Acc: 98.26%)

Epoch 10/10


Validation: 100%|████████████████████████████████████████████████████████████████████| 187/187 [05:50<00:00,  1.87s/it]

Train Loss: 0.1851 | Train Acc: 96.58%
Val   Loss: 0.0948 | Val   Acc: 98.49%
✅ New best model saved (Val Acc: 98.49%)


In [1]:
from PIL import Image
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch.nn as nn
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MTCNN and FaceNet
mtcnn = MTCNN(image_size=160, margin=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Classifier Head
class ClassifierHead(nn.Module):
    def __init__(self, embedding_dim=512, num_classes=10):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.classifier(x)


C:\Users\monoj\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
label_map = val_dataset.label_map
inv_label_map = {v: k for k, v in label_map.items()}


In [11]:
# Make sure num_classes matches the training setup
classifier = ClassifierHead(num_classes=len(label_map)).to(device)
classifier.load_state_dict(torch.load('best_classifier.pth', map_location=device))
classifier.eval()


ClassifierHead(
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=250, bias=True)
  )
)

In [13]:
def predict_image(image_path):
    img = Image.open(image_path).convert('RGB')
    
    # Detect face
    face = mtcnn(img)
    if face is None:
        print("❌ No face detected in the image.")
        return
    
    # Extract embedding
    with torch.no_grad():
        embedding = resnet(face.unsqueeze(0).to(device))

        # Predict
        output = classifier(embedding)
        _, pred = torch.max(output, 1)
        predicted_label = pred.item()
    
    print(f"✅ Predicted: {inv_label_map[predicted_label]}")


In [14]:
predict_image('Task_B/val/Bill_Readdy/distortion/Bill_Readdy_0001_noisy.jpg')


✅ Predicted: Bill_Readdy


In [15]:
predict_image('Task_B/val/Gloria_Gaynor/Gloria_Gaynor_0001.jpg')

✅ Predicted: Gloria_Gaynor
